In [1]:
#library addition
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_validate, cross_val_predict
from pathlib import Path

In [2]:
#data read
clinicalInput = pd.read_excel(Path("Balanced Data Set.xlsx"))

In [3]:
#determine the number of classes(labels)
label_encoder = LabelEncoder().fit(clinicalInput.Label)
labels = label_encoder.transform(clinicalInput.Label)
classes = list(label_encoder.classes_)

In [4]:
#data preparation
clinicalOutput = clinicalInput["Label"]
clinicalInput = clinicalInput.drop(["Patient ID", "Patient age quantile","Label"],axis=1)

In [5]:
#determine number of features and classes
nb_features = 18
nb_classes = len(classes)
print(nb_classes)

2


In [6]:
#Standardization of train data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(clinicalInput.values)
clinicalInput = scaler.transform(clinicalInput.values)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization
from tensorflow.keras import layers

from keras import backend as K

In [8]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [9]:
from tensorflow.keras.utils import to_categorical
clinicalOutput = to_categorical(clinicalOutput, num_classes=nb_classes)

print(clinicalInput)
print(clinicalOutput)
#clinicalInput = np.array(clinicalInput).reshape(600,18,1)

#build cnn model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Dropout, MaxPooling1D, Flatten, SimpleRNN, BatchNormalization


[[ 2.33733862e-01 -2.43875529e-02 -5.23575242e-01 ...  1.10891428e+00
  -1.44002434e-03  1.28848209e-03]
 [-1.57095390e+00 -7.74766806e-01  1.42723113e+00 ...  6.47827206e-01
  -9.61677873e-01 -2.63395334e-01]
 [-7.48564199e-01 -5.87171792e-01 -4.35474300e-01 ... -2.74346787e-01
  -9.16572703e-01 -8.95058915e-01]
 ...
 [ 7.13460531e-01  1.10118155e+00 -4.98403540e-01 ...  1.86740229e-01
   1.42889330e+00  2.26325887e+00]
 [-3.23857636e+00 -2.77577827e+00 -1.78215992e+00 ... -1.65760770e+00
  -8.26362461e-01 -1.08455795e+00]
 [ 6.90616716e-01  5.38397281e-01 -9.13736516e-01 ... -1.19652078e+00
  -4.65521542e-01  1.78769172e-01]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [18]:
def create_network():
    model = Sequential()
    model.add(SimpleRNN(512,input_shape=(nb_features,1)))
    model.add(Activation("relu"))
    model.add(Dropout(0.25))
    model.add(BatchNormalization())
    model.add((Flatten()))
    model.add(Dense(2048, activation="relu"))
    model.add(Dense(1024, activation="relu"))
    model.add(Dense(nb_classes, activation="softmax")) 
    model.summary()

    from tensorflow.keras.optimizers.schedules import ExponentialDecay
    from tensorflow.keras.optimizers import SGD
    initial_learning_rate = 0.1
    decay_steps = 10000
    decay_rate = 0.1

    lr_schedule = ExponentialDecay(
        initial_learning_rate,
        decay_steps=decay_steps,
        decay_rate=decay_rate,
        staircase=True)

    opt = SGD(learning_rate=lr_schedule)

    model.compile(loss="binary_crossentropy", optimizer = opt, metrics=["accuracy",f1_m,precision_m,recall_m])
    return model

In [19]:
neural_network = KerasClassifier(build_fn=create_network, 
                                 epochs=250)




C:\Users\akims\AppData\Local\Temp\ipykernel_26924\1581329954.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  neural_network = KerasClassifier(build_fn=create_network,


In [20]:
#from sklearn.preprocessing import LabelBinarizer
#lb = LabelBinarizer()
#clinicalOutput = np.array([number[0] for number in lb.fit_transform(clinicalOutput)])

In [24]:
results = cross_validate(neural_network, clinicalInput, clinicalOutput,cv=2,scoring=("accuracy","f1","recall","precision"))


import matplotlib.pyplot as plt
plt.plot(results["test_accuracy"],color="c")
plt.plot(results["test_f1"],color="m")
plt.plot(results["test_recall"],color="y")
plt.plot(results["test_precision"],color="k")
plt.title("Model Information (RNN)")
plt.ylabel("Model Performance")
plt.xlabel("Number of Folds")
plt.legend(["Accuracy","F1-Score","Recall","Precision"], loc="lower right")
plt.show()
print ("neural_network: ",neural_network,"\n clinicalInput: ",clinicalInput,"\n clinicalOutput: ",clinicalOutput)
#Determine the prediction
y_pred = cross_val_predict(neural_network, clinicalInput, clinicalOutput, cv=2)


#Provide AUC score
from sklearn.metrics import roc_auc_score
print("Accuracy result: ", np.mean(results["test_accuracy"]))
print("Recall result: ", np.mean(results["test_recall"]))
print("Precision result: ", np.mean(results["test_precision"]))
print("F1 result: ", np.mean(results["test_f1"]))
##print("ROC: ", roc_auc_score(clinicalOutput, y_pred))



Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_8 (SimpleRNN)    (None, 512)               263168    
                                                                 
 activation_8 (Activation)   (None, 512)               0         
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 batch_normalization_8 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 flatten_8 (Flatten)         (None, 512)               0         
                                                                 
 dense_24 (Dense)            (None, 2048)              1050624   
                                                      

ValueError: 
All the 2 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\akims\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\akims\anaconda3\lib\site-packages\keras\wrappers\scikit_learn.py", line 236, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "C:\Users\akims\anaconda3\lib\site-packages\keras\wrappers\scikit_learn.py", line 164, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\akims\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:

In [ ]:
results

In [22]:
print(clinicalOutput[:10])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [17]:
len(y_pred)

600